# 1. 합성곱 신경망의 구성 요소

#### 합성곱 - 입력 데이터 전체에 가중치를 적용하는 것이 아니라 일부에 가중치 곱한다.

#### 10개의 입력을 가진 합성곱의 뉴런은 8개의 출력을 만든다.(가중치가 3개일 경우)

#### 필터 - 뉴런이 입력위를 이동하면서 출력을 만들기 때문에 뉴런을 필터라 한다.

## 2. 케라스의 합성곱 층

#### 케라스의 합성곱은 keras.layers 패키지아래 Conv2D 클래스로 제공함.

In [2]:
from  tensorflow import keras
#10 - 필터의 갯수, kernel_size : 커넷의 크기, crivateoin: 활성화 함수
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu')

<Conv2D name=conv2d_1, built=False>

### (1) 패딩
#### 입력과 턱성맵의 크기를 동일하게 만들기 위해서 입력주위에 9으로 채워주는 것을 패이(세임패딩)이라 한다.


## 패딩을 사용하는 이유
#### 패딩없이 합성곱을 한다면 위쪽 모서리의 3은 커녕 가중치 계산을 한번만 참여
#### 반면 다른 원소들은 2번 이상 커널 계산에 참여, 각 모서리의 중요한 정보가 특성맵에 전달되지 않을 가능성이 높음

In [7]:
# 케라스에서 패딩을 지정하는 방법 : padding 속성
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same')     

<Conv2D name=conv2d_4, built=False>

### 스트라이드
#### 합성곱(필터)의 이동 크기를 지정. 기본값은 1

In [8]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same', strides=1)

<Conv2D name=conv2d_5, built=False>

### 풀링(Pooling)

#### 합성곱 층에서 만든 특성맵의 가로,세로 크기를 줄이는 역할을 수행
#### 특성맵의 갯수는 줄지 않는다.
#### 풀링에는 가중치가 없고 필터를 적용한 영역에서 가장 큰 값을 고르거나(최대풀링) 평균값을 계산(평균 풀링)
#### 풀링은 합성곱층과 구분하기 위해 풀링층이라 부른다.

In [11]:
# 케라스에서 풀링의 구현. strides나 padding매개변수 제공
keras.layers.MaxPooling2D(2)  # 최대 풀링. 매개변수2: 풀링의 크기 지정

<MaxPooling2D name=max_pooling2d_2, built=False>

#### 평균 풀링 클래스 : AveragePooling2D

## 3. 합성곱 신경망을 이용한 이미지 분류

### (1) MNIST 데이터 불러오기

In [14]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
# 흑백이미지의 경우 채널 차원이 없는 2차원 배열이지만, Conv2D층을 사용하기 위해 마지막에 채널 차원을 추가해야 함.
train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

### (2) 합성곱 신경망 만들기

In [18]:
# Sequential 클래스에 합성곱층 Conv2D 추가
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(28, 28, 1)))

#### 합성곱 충은 32개의 필터 사용, 커널의 크기(3,3),relu 활성화함수, 세임패딩 사용

#### 풀링층 추가 

In [19]:
model.add(keras.layers.MaxPooling2D(2))

#### * 패션 MNIST 입력 이미지가 (28,28)이고, 세임패딩을 적용했기 때문에 합성곱층의 출력은 입력과 동일한 크기임.
#### * (2,2) 풀링을 정용했으므로 특성맵의 절반으로 줄어서 (14,14,32)가 될것임.

### (3) 두번째 합성곱층, 풀링층 추가

In [20]:
# 필터의 갯수를 64개로 늘림
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))          

#### 세임패딩을 사용했으므로 입력의 크기와 마찬가지로 14x14이고, 필터는 64이므로 (14,14,64)가 됨
#### 풀링층을 통과한 최종 맵은 (7, 7, 64) 특성맵이 산출됨.

### (4) 3차원 특성맵을 일렬로 펼친다.

In [21]:
# Flatten, Dense, Dense, 출력층으로 구성
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10, activation='softmax'))

In [22]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │       313,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333,526 (1.27 MB)

 Trainable params: 333,526 (1.27 MB)

 Non-trainable params: 0 (0.00 B)